### A larger more pythonic example

A version of @inconvergent's wind rendering to p55py.
https://github.com/inconvergent/wind

In [1]:
from p55py import P55Wrapper
from IPython.display import display
import ipywidgets as widgets
import webcolors

In [2]:
# some color constants (we'll be able to override one from the panel)

BACK = (255,255,255,255)
FRONT = (0, 0, 0, 255)
LIGHT = (0, 100, 0, 255)
CYAN = (0, 100, 100, 255)
BLUE = (0, 0, 200, 255)
RED = (128, 0, 0, 255)

FRONTCOLORS = [FRONT, LIGHT, CYAN, BLUE, RED]

In [3]:
# simulation constants (we'll expose two in the panel)
NMAX = 10**6
SIZE = 200
ONE = 1.5/SIZE
LINEWIDTH = ONE*1.1

INIT_NUM = 4

STP = ONE*3

ANGLE_STP = 0.5
ANGLE_LOCAL_STP = 0.8

In [4]:
from modules.wind import Wind

# setup to initialize
def setup_wind(self):
    global ANGLE_STP, ANGLE_LOCAL_STP
    self.wind = Wind(NMAX, SIZE, STP, ANGLE_STP, ANGLE_LOCAL_STP)
    self.wind.rnd_seed(INIT_NUM)

# step updates the model
def step_wind(self):
    res = self.wind.step()

# render draws the model (idempotent)
def render_wind(self):
    self.fill(225)
    self.cur_outline = self.cur_fill
    self.rectangle(0,0,320,320)
    self.fill(0)

    xy = self.wind.xy
    n = self.wind.n
    r = self.wind.r

    for i,x in enumerate(xy[:n,:]):
        self.cur_fill = FRONTCOLORS[r[i]]
        self.cur_outline = FRONTCOLORS[r[i]]
        self.ellipse(int(320*x[0]), int(320*x[1]), 2, 2)

    for i,p in enumerate(self.wind.p[:n]):
        if p>-1:
            self.cur_fill = FRONTCOLORS[r[i]]
            self.cur_outline = FRONTCOLORS[r[i]]
            source = 320 * xy[p,:].flatten()
            dest = 320 * xy[i,:].flatten()
            self.line(source[0], source[1], dest[0], dest[1])


### With no panel, straightforward to use

In [5]:
wrapper = P55Wrapper(setupfn=setup_wind, stepfn=step_wind, renderfn=render_wind)
wrapper.play.max = 1000
wrapper.redrawFreq = 5  # draw only every 5th step
display(wrapper.widget)

### Wiring up a panel is currently cumbersome, but doable
Note that in addition to the model parameters above, this panel also exposes some more generic settings:
 * the current random seed
 * whether the seed is auto-incremented each run (disable to manually return to known good seed)
 * the draw frequency (updates per render)

In [6]:
def wrapped_wind_settings(wrapper):
    def settings_updated(randomSeed, incSeed, drawFreq, ASTP, ALOCAL, color):
        global FRONTCOLORS
        global ANGLE_STP
        global ANGLE_LOCAL_STP
        wrapper.randomSeed = randomSeed
        wrapper.incSeedOnStop = incSeed
        wrapper.redrawFreq = drawFreq
        ANGLE_STP = ASTP
        ANGLE_LOCAL_STP = ALOCAL
        t = webcolors.html5_parse_legacy_color(color)
        FRONTCOLORS[0] = (t[0], t[1], t[2], 255)
        return None
    return settings_updated

In [7]:
wrapper = P55Wrapper(setupfn=setup_wind, stepfn=step_wind, renderfn=render_wind)
settings_callback = wrapped_wind_settings(wrapper)
cp = widgets.ColorPicker()
seedWidget = widgets.IntText(wrapper.randomSeed)
incSeedWidget = widgets.Checkbox(description='Inc Seed on Stop', value=wrapper.incSeedOnStop)
rw = widgets.IntSlider(value=1,min=1,max=20)
asw = widgets.FloatSlider(value=0.5,min=0,max=2)
alsw = widgets.FloatSlider(value=0.8,min=0,max=2)
w = widgets.interactive(settings_callback, randomSeed=seedWidget, incSeed=incSeedWidget, drawFreq=rw, ASTP=asw, ALOCAL=alsw, color=cp)
wrapper.addSettingsPane(w, seedWidget)
wrapper.play.max = 1000
wrapper.redrawFreq = 5
display(wrapper.widget)